In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 13 06:17:34 2019

@author: User
"""

import numpy as np
import pandas as pd

data = pd.read_csv('train1.csv')
test_data = pd.read_csv('test.csv')

def process_data(data, test_data):
    df = pd.concat([data, test_data])
    TARGET = df['TARGET']
    df = df.drop('TARGET', axis=1)
    df = pd.get_dummies(df, dummy_na=True)
    df = df.fillna(0)
    df['TARGET'] = TARGET
    
    from sklearn.preprocessing import PolynomialFeatures
    poly_features = df[[
            'age', 'duration', 'default', 'campaign', 'loan']]
    poly_transformer = PolynomialFeatures(degree = 3)
    poly_transformer.fit(poly_features)
    poly_features = poly_transformer.transform(poly_features)
    
    poly_features = pd.DataFrame(poly_features, columns = poly_transformer.get_feature_names([
            'age', 'duration', 'default', 'campaign', 'loan']))
    
    ID = np.array(df['ID'])
    poly_features['ID'] = ID
    app_poly = df.merge(poly_features, on='ID', how='left')
    
    return app_poly

df = process_data(data, test_data)


def modeling(df):
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    train_x = train_df.drop(['TARGET', 'ID'], axis=1)
    train_y = train_df['TARGET']
    
    test_x = test_df.drop(['TARGET', 'ID'], axis=1)
    
    from sklearn.model_selection import train_test_split
    train_x_in, valid_x, train_y_in, valid_y = train_test_split(train_x, train_y, test_size = 0.2)

    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression()
    model.fit(train_x_in, train_y_in)
    prediction_all = model.predict_proba(valid_x)
    prediction = prediction_all[:,1]
    
    from sklearn.metrics import roc_auc_score
    score = roc_auc_score(np.array(valid_y), prediction)
    print('score: ', score)
    
    prediction_test = model.predict_proba(test_x)[:,1]
    sub = pd.DataFrame()
    sub['TARGET'] = prediction_test
    sub['ID'] = test_df['ID'].reset_index(drop=True)
    sub.to_csv('submission.csv', index=False)
    
    return  model, sub

model, sub = modeling(df)

from sklearn.externals import joblib
joblib.dump(model, 'train1.model')


'\nCreated on Mon May 13 06:17:34 2019\n\n@author: User\n'

c:\myprogram\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


score:  0.7030754403942809


c:\myprogram\python\python37\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


['train1.model']